In [7]:
%%capture
%pip install numpy
!pip install torch
%pip install pandas

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from torch.utils.data import Dataset, random_split
from torch.utils.data import DataLoader

In [14]:
full_dataset = pd.read_csv('./titanic/train.csv')

y = full_dataset['Survived']
X_unprocessed = full_dataset[['Pclass', 'Sex', 'SibSp', 'Parch', 'Age', 'Fare', 'Embarked']]

X = pd.get_dummies(X_unprocessed).astype(float)

X = torch.tensor(X.values).float()
y = torch.tensor(y).long()

tensor([[3., 1., 0.,  ..., 0., 0., 1.],
        [1., 1., 0.,  ..., 1., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [3., 1., 2.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        [3., 0., 0.,  ..., 0., 1., 0.]])

In [ ]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size 


seed = torch.Generator().manual_seed(42)
train_dataset, test_dataset= random_split(torch.utils.data.TensorDataset(X,y), [train_size, test_size], generator=seed)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


linear_model = torch.nn.Sequential(
    torch.nn.Linear(2,100), 
    torch.nn.ReLU(),
    torch.nn.Linear(100,2),
)


loss_fn = torch.nn.CrossEntropyLoss()
lr = 1e-2
opt = torch.optim.SGD(linear_model.parameters(), lr=lr, weight_decay=1e-4)



epochs = 100
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        opt.zero_grad() # zero out the gradients

        z = linear_model(batch_x) # compute z values
        loss = loss_fn(z,batch_y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')


